## Example of ways to calc requested layers from a DSM & DTM pair  

input:  
    `*dsm.tif`  
    `*dtm.tif `  
requested layers:  
    `chm.tif`  : canopy height (m)  
    `cc.tif` : canopy cover (proportion, 0-1)  
    `rugosity.tif` : rugosity (m)

In [7]:
import numpy as np
import sys
sys.path.append('/home/pmontesa/code/pygeotools')
from pygeotools.lib import iolib#, malib, geolib, filtlib, warplib

In [ ]:
# Get list of files
dsm_fn = 'file path to DSM'
dtm_fn = 'file path to DTM'
fn_list = [dsm_fn, dtm_fn]

In [ ]:
# Example of file open with gdal
dsm_ds = gdal.Open(str(dsm_fn))
dtm_ds = gdal.Open(str(dtm_fn))

In [ ]:
# Example to get the masked arrays
dsm_ma, dtm_ma  = [iolib.fn_getma(fn) for fn in fn_list]

In [ ]:
# Calc CHM
chm_ma = dsm_ma - dtm_ma

#### This moving window function can be used for a few calcs

In [ ]:
def moving_window(arr, func=np.mean, window_size=3):
    """moving_window(array, window_size, func=mean)
    """
    from scipy.ndimage.filters import generic_filter
    print "\tCheck array type INPUT to filter: %s" %(arr.dtype)
    return generic_filter(arr, func, size=window_size)

In [ ]:
# Calc canopy cover from CHM
# Lidar canopy cover fraction using CHM 3x3 pixel window, sliding by 1 pixel
sz_window = 3
threshold_height = 2 # check with team - what is correct threshold?
canopy_ma = np.where(chm_ma < threshold_height, 0, 1)
cc_ma = moving_window(canopy_ma, func=np.sum, window_size=sz_window) / (sz_window * sz_window)

In [ ]:
# Calc rugosity from CHM
# Lidar rugosity using DSM 3x3 pixel window, sliding by 1 pixel
rug_ma = moving_window(chm_ma, np.std, sz_window)

In [ ]:
# Not requested - but an alternative to rugosity
# Calc roughness of CHM
import osgeo
from osgeo import gdal
roughness_ds = gdal.DEMProcessing('', dem_ds, 'roughness', format='MEM')

In [ ]:
#Given input dataset, return a masked array for the input band
def ds_getma(ds, bnum=1):
    """Get masked array from input GDAL Dataset
    Parameters
    ----------
    ds : gdal.Dataset 
        Input GDAL Datset
    bnum : int, optional
        Band number
    
    Returns
    -------
    np.ma.array    
        Masked array containing raster values
    """
    b = ds.GetRasterBand(bnum)
    return b_getma(b)
